* [#49](https://github.com/salgo60/SCB-Wikidata/issues/49)
* [Notebook Regeringskansliet_49.ipynb](https://github.com/salgo60/SCB-Wikidata/blob/main/notebook/Regeringskansliet_49.ipynb)
* [links_regeringen_v1_2026_01_07.html](http://salgo60.github.io/SCB-Wikidata/notebook/resultsRegeringen/links_regeringen_v1_2026_01_07.html)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2026-01-07 20:22:04


In [2]:

SCRIPT_NAME = "Regeringskansliet_49.ipynb"
SCRIPT_URL = (
    "https://github.com/salgo60/SCB-Wikidata/"
    "blob/master/notebook/Regeringskansliet_49.ipynb"
) 


In [3]:
def read_domains(file_path):
    print(f"[DEBUG] Reading domains from: {file_path}")
    df = pd.read_csv(file_path, header=0)   # <- skip header row
    domains_list = df.iloc[:, 0].dropna().unique().tolist()
    print(f"[DEBUG] Found {len(domains_list)} domains.")
    return domains_list


In [4]:
import requests

def fetch_sitematrix_df():
    url = "https://meta.wikimedia.org/w/api.php"
    params = {
        "action": "sitematrix",
        "format": "json"
    }
    headers = {
        "User-Agent": "salgo60-language-fetcher/1.0 (salgo60@msn.com)"
    }

    print("[DEBUG] Fetching sitematrix…")
    r = requests.get(url, params=params, headers=headers)
    r.raise_for_status()

    if "application/json" not in r.headers.get("Content-Type", ""):
        raise ValueError("Server returned non-JSON response")

    data = r.json()["sitematrix"]

    rows = []

    # --- language-specific sites ---
    for key, lang_block in data.items():
        if not key.isdigit():
            continue  # skip "count", "specials"

        lang_code = lang_block.get("code")
        lang_name = lang_block.get("name")

        for site in lang_block.get("site", []):
            rows.append({
                "lang_code": lang_code,
                "lang_name": lang_name,
                "project": site.get("project"),
                "url": site.get("url"),
                "dbname": site.get("dbname"),
                "site_name": site.get("sitename"),
                "closed": site.get("closed", False)
            })

    # --- special wikis (Wikidata, Commons, Meta, etc.) ---
    for site in data.get("specials", []):
        rows.append({
            "lang_code": "special",
            "lang_name": "special",
            "project": site.get("project"),
            "url": site.get("url"),
            "dbname": site.get("dbname"),
            "site_name": site.get("sitename"),
            "closed": site.get("closed", False)
        })

    return pd.DataFrame(rows)


In [5]:
import requests
import pandas as pd


HEADERS = {
    "User-Agent": "salgo60-language-fetcher/2.0 (https://github.com/salgo60)"
}


df_lang_fetch = fetch_sitematrix_df()
df_lang_fetch["closed"] = df_lang_fetch["closed"].fillna(False).astype(bool)

df_lang_wikipedia = df_lang_filtered = df_lang_fetch[
    (df_lang_fetch["site_name"] == "Wikipedia") &
    (df_lang_fetch["lang_name"].str.lower() != "special")]

#df_lang_wikipedia.to_csv("test.csv")
df_lang_wikipedia.info()

[DEBUG] Fetching sitematrix…
<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 937
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lang_code  184 non-null    object
 1   lang_name  183 non-null    object
 2   project    0 non-null      object
 3   url        184 non-null    object
 4   dbname     184 non-null    object
 5   site_name  184 non-null    object
 6   closed     184 non-null    bool  
dtypes: bool(1), object(6)
memory usage: 10.2+ KB


In [6]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)



Current Working Directory: /Users/salgo/Documents/GitHub/SCB-Wikidata/notebook


In [7]:
# -----------------------------------------------------------
# Fetch exturlusage entries for one lang/domain
# -----------------------------------------------------------
def fetch_exturlusage(lang, domain):
    base = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "exturlusage",
        "euquery": domain,
        "eulimit": "max"
    }
    while True:
        r = session.get(base, params=params, timeout=10)
        try:
            data = r.json()
        except ValueError:
            print(f"[WARN] {lang}: JSON decode failed")
            break

        for item in data.get("query", {}).get("exturlusage", []):
            yield {
                "lang": lang,
                "page_title": item.get("title"),
                "url": item.get("url"),
                "wiki_link": f"https://{lang}.wikipedia.org/wiki/{item.get('title').replace(' ', '_')}"
            }

        if "continue" not in data:
            break
        params.update(data["continue"])
        time.sleep(0.3)

In [8]:
import os
import time
import random
import requests
import pandas as pd
from urllib.parse import urlparse
from tqdm.notebook import tqdm
file_path_domain = "sources/domains_Regeringskansliet.csv"
domains = read_domains(file_path_domain)
print(domains)


[DEBUG] Reading domains from: sources/domains_Regeringskansliet.csv
[DEBUG] Found 1 domains.
['regeringen.se']


In [9]:
# -----------------------------------------------------------
# Fetch exturlusage entries for one lang/domain
# -----------------------------------------------------------
def fetch_exturlusage(lang, domain):
    base = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "exturlusage",
        "euquery": domain,
        "eulimit": "max"
    }
    while True:
        r = session.get(base, params=params, timeout=10)
        try:
            data = r.json()
        except ValueError:
            print(f"[WARN] {lang}: JSON decode failed")
            break

        for item in data.get("query", {}).get("exturlusage", []):
            yield {
                "lang": lang,
                "page_title": item.get("title"),
                "url": item.get("url"),
                "wiki_link": f"https://{lang}.wikipedia.org/wiki/{item.get('title').replace(' ', '_')}"
            }

        if "continue" not in data:
            break
        params.update(data["continue"])
        time.sleep(0.3)

In [10]:

# -------------------------
# Session & helpers
# -------------------------
session = requests.Session()
session.headers.update({"User-Agent": "SCB-LinkAudit/1.0 (https://www.scb.se/)"})

# we need some filtering  

print("Antal Språk:",len(df_lang_wikipedia ))
results = []
for _, row in df_lang_wikipedia.iterrows():
    lang = row["lang_code"]
    url  = row["url"]
    lang_name = row["lang_name"]
    before = len(results)
    #print(lang, url, lang_name,domains)
    for entry in fetch_exturlusage(lang, domains):
        results.append(entry)     
    after = len(results) 
    links = after-before
    print(lang, url, lang_name," - ", links)
    


Antal Språk: 184
aa https://aa.wikipedia.org Qafár af  -  0
ace https://ace.wikipedia.org Acèh  -  0
af https://af.wikipedia.org Afrikaans  -  3
ak https://ak.wikipedia.org None  -  0
ami https://ami.wikipedia.org Pangcah  -  0
an https://an.wikipedia.org aragonés  -  0
ast https://ast.wikipedia.org asturianu  -  6
av https://av.wikipedia.org авар  -  0
avk https://avk.wikipedia.org Kotava  -  0
ay https://ay.wikipedia.org Aymar aru  -  0
bar https://bar.wikipedia.org Boarisch  -  1
bbc https://bbc.wikipedia.org Batak Toba  -  0
bcl https://bcl.wikipedia.org Bikol Central  -  4
bi https://bi.wikipedia.org Bislama  -  0
bm https://bm.wikipedia.org bamanankan  -  0
bo https://bo.wikipedia.org བོད་ཡིག  -  0
br https://br.wikipedia.org brezhoneg  -  0
bs https://bs.wikipedia.org bosanski  -  1
btm https://btm.wikipedia.org Batak Mandailing  -  0
bug https://bug.wikipedia.org Basa Ugi  -  0
bxr https://bxr.wikipedia.org буряад  -  0
cbk-zam https://cbk-zam.wikipedia.org Chavacano de Zamboan

In [11]:
df_regeringen = pd.DataFrame(results)
df_regeringen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9628 entries, 0 to 9627
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lang        9628 non-null   object
 1   page_title  9628 non-null   object
 2   url         9628 non-null   object
 3   wiki_link   9628 non-null   object
dtypes: object(4)
memory usage: 301.0+ KB


In [12]:
import pandas as pd

# --- Stats ---
total_links = len(df_regeringen)
total_unique_links = df_regeringen['url'].nunique()
num_languages = df_regeringen['lang'].nunique()
langs_sorted = df_regeringen['lang'].value_counts()

print("Total links:", total_links)
print("Total unique links:", total_unique_links)
print("Number of languages:", num_languages)
print("\nLanguages with most links:")
print(langs_sorted.to_string())


Total links: 9628
Total unique links: 5594
Number of languages: 36

Languages with most links:
lang
sv        6352
en        1971
no         387
fi         180
de         149
es         127
id          66
ro          59
zh          50
da          46
it          42
ja          37
pl          36
vi          21
ms          17
nn          11
nl          11
ig           8
sq           7
fo           7
simple       7
ha           6
ast          6
gl           4
is           4
bcl          4
af           3
sw           2
pcm          1
sco          1
bs           1
min          1
bar          1
tt           1
eu           1
ff           1


In [22]:
# --- Stats ---
total_links = len(df_regeringen)
total_unique_links = df_regeringen["url"].nunique()
langs_with_hits = sorted(df_regeringen["lang"].unique())

num_languages_found = len(langs_with_hits)
num_languages_checked = len(df_lang_wikipedia)        # alla språk som genomsöktes
num_languages_found = df_regeringen['lang'].nunique()


In [23]:
num_languages_checked 

184

In [38]:
import requests
from requests.exceptions import RequestException
from tqdm import tqdm

SOFT_404_PHRASES = [
    "Sidan kan inte hittas",
    "sidan tagits bort",
    "felaktig adress",
    "kontakta registrator",
    "Regeringskansliets arkiv",
]

ROOT_URLS = {
    "https://www.regeringen.se/",
    "http://www.regeringen.se/",
}

def check_regeringen_url(url, timeout=15):
    try:
        r = requests.get(
            url,
            allow_redirects=True,
            timeout=timeout,
            headers={
                "User-Agent": "LinkChecker/1.0 (research; contact info)"
            }
        )
    except RequestException as e:
        return {
            "url": url,
            "status": "error",
            "reason": str(e),
        }

    final_url = r.url
    status_code = r.status_code
    text = r.text or ""

    # 1. Hard HTTP error
    if status_code >= 400:
        return {
            "url": url,
            "status": "dead",
            "reason": f"HTTP {status_code}",
            "final_url": final_url,
        }

    # 2. Redirected to site root
    if final_url.rstrip("/") + "/" in ROOT_URLS:
        return {
            "url": url,
            "status": "dead",
            "reason": "redirect_to_root",
            "final_url": final_url,
        }

    # 3. Soft 404 detection
    lowered = text.lower()
    for phrase in SOFT_404_PHRASES:
        if phrase.lower() in lowered:
            return {
                "url": url,
                "status": "dead",
                "reason": "soft_404",
                "final_url": final_url,
            }

    # Otherwise OK
    return {
        "url": url,
        "status": "ok",
        "final_url": final_url,
    }


In [ ]:
from tqdm import tqdm  
results = []
checked = set()

for url in tqdm(
    df_regeringen["url"],
    total=len(df_regeringen),
    desc="Checking regeringen.se links",
    unit="link",
):
    if url in checked:
        continue
    result = check_regeringen_url(url)
    results.append(result)
    checked.add(url)
    time.sleep(0.3)  


Checking regeringen.se links:   4%|▏     | 361/9628 [03:37<1:12:11,  2.14link/s]

In [41]:
#results = df_regeringen["url"].apply(check_regeringen_url)
#status_df = pd.DataFrame(results.tolist())
#df_regeringen = df_regeringen.join(status_df.set_index("url"), on="url")

In [ ]:
df_regeringen.info()

In [24]:
from datetime import date
import os

# Sätt datum
today = date.today().strftime("%Y_%m_%d")

# Se till att katalogen finns
os.makedirs("resultsRegeringen", exist_ok=True)

# Bygg filnamn
outfile = f"resultsRegeringen/links_Regeringen_v1_{today}.csv"

# Exportera
df_regeringen.to_csv(outfile, index=False, encoding="utf-8")

print(f"[OK] Exported {len(df_regeringen)} rows to {outfile}")


[OK] Exported 9628 rows to resultsRegeringen/links_Regeringen_v1_2026_01_08.csv


In [32]:
from pathlib import Path
from datetime import date, datetime
from urllib.parse import quote
import pandas as pd


def save_sortable_html_df_regeringen(
    df,
    out_dir="resultsRegeringen",
    domains=None,
    issue_url="https://github.com/salgo60/SCB-Wikidata/issues/49",
):
    out_dir = Path(out_dir)
    out_dir.mkdir(exist_ok=True)

    today = date.today().strftime("%Y_%m_%d")
    out_path = out_dir / f"links_regeringen_v1_{today}.html"
    rerun_ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    domains = domains or []

    # --- Förbered data ---
    df = df.copy()

    # Wikipedia: ikon + titel (byggd från lang + page_title)
    if {"lang", "page_title"}.issubset(df.columns):
        df["page_title"] = df.apply(
            lambda r: (
                f'<a href="https://{r["lang"]}.wikipedia.org/wiki/{quote(str(r["page_title"]))}" '
                f'target="_blank" title="Wikipedia ({r["lang"]})">'
                f'<i class="fa-brands fa-wikipedia-w" style="margin-right:6px;"></i>'
                f'{r["page_title"]}</a>'
                if pd.notna(r["lang"]) and pd.notna(r["page_title"])
                else r["page_title"]
            ),
            axis=1,
        )

    # Externa länkar
    for col in ["Wikipedia-länk", "Extern länk", "url"]:
        if col in df.columns:
            df[col] = df[col].apply(
                lambda x: f'<a href="{x}" target="_blank">{x}</a>' if pd.notna(x) else ""
            )

    # --- HTML-tabell ---
    html_table = df.to_html(
        classes="pivot",
        border=0,
        escape=False,  # krävs för HTML-länkar
        index=False,
    )

    # --- CSS ---
    css = """
    <style>
      body {
        font-family: Arial, sans-serif;
        margin: 20px;
      }
      table.pivot {
        border-collapse: collapse;
        width: 100%;
        font-size: 12px;
      }
      table.pivot th, table.pivot td {
        border: 1px solid #999;
        padding: 6px 8px;
        text-align: left;
        vertical-align: top;
        white-space: normal;
      }
      table.pivot th {
        cursor: pointer;
        background: #f2f2f2;
      }
      table.pivot th:hover {
        background: #e2e2e2;
      }
      table.pivot thead th {
        position: sticky;
        top: 0;
        background: #f2f2f2;
        z-index: 2;
      }
      table.pivot th::after {
        content: "";
        float: right;
        opacity: 0.4;
      }
      table.pivot th.sorted-asc::after {
        content: " ▲";
      }
      table.pivot th.sorted-desc::after {
        content: " ▼";
      }
      table.pivot td a {
        color: #0645ad;
        text-decoration: none;
      }
      table.pivot td a:hover {
        text-decoration: underline;
      }
      .meta {
        background: #f8f8f8;
        border: 1px solid #ccc;
        padding: 12px;
        margin-bottom: 20px;
        font-size: 13px;
      }
      .meta h2 {
        margin-top: 0;
      }
    </style>
    """

    # --- JavaScript (sortering) ---
    js = """
    <script>
    document.addEventListener('DOMContentLoaded', () => {
        document.querySelectorAll("table.pivot th").forEach((header, colIndex) => {
            header.addEventListener("click", () => {
                const table = header.closest("table");
                const tbody = table.querySelector("tbody");
                const rows = Array.from(tbody.querySelectorAll("tr"));
                const asc = !header.classList.contains("sorted-asc");

                rows.sort((a, b) => {
                    const A = a.children[colIndex].innerText.trim();
                    const B = b.children[colIndex].innerText.trim();
                    const numA = parseFloat(A.replace(",", "."));
                    const numB = parseFloat(B.replace(",", "."));
                    if (!isNaN(numA) && !isNaN(numB)) {
                        return asc ? numA - numB : numB - numA;
                    }
                    return asc ? A.localeCompare(B) : B.localeCompare(A);
                });

                table.querySelectorAll("th").forEach(th =>
                    th.classList.remove("sorted-asc", "sorted-desc")
                );
                header.classList.add(asc ? "sorted-asc" : "sorted-desc");
                rows.forEach(row => tbody.appendChild(row));
            });
        });
    });
    </script>
    """

    # --- Metadata ---
    meta_html = f"""
    <div class="meta">
      <h2>Metadata</h2>
      <p><strong>Rerun:</strong> {rerun_ts}</p>
      <p><strong>Issue:</strong>
         <a href="{issue_url}" target="_blank">{issue_url.split("/")[-1]}</a>
      </p>
      <p><strong>Datakällor:</strong><br>
         Wikidata<br>
         MediaWiki API – exturlusage
      </p>
    </div>
    """

    # --- Slutlig HTML ---
    html = f"""
    <html>
    <head>
      <meta charset="utf-8">
      <title>regeringen.se links in Wikipedia</title>
      <link rel="stylesheet"
            href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css">
      {css}
    </head>
    <body>
      <h1>Wikipedia → regeringen v1</h1>
      {meta_html}
      <p>Sorterbar tabell. Klicka på kolumnrubriker för sortering.</p>
      {html_table}
      {js}
    </body>
    </html>
    """

    out_path.write_text(html, encoding="utf-8")
    print(f"✅ HTML skapad: {out_path}")


In [33]:

save_sortable_html_df_regeringen(df_regeringen)


✅ HTML skapad: resultsRegeringen/links_regeringen_v1_2026_01_08.html


In [29]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2026-01-08 10:20:45
Total time elapsed: 838 minutes 40.79 seconds
